In [7]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import nltk

In [8]:
## downloading reuters dataset
nltk.download("reuters")

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\E14\AppData\Roaming\nltk_data...


True

In [9]:
## importing the reuters dataset
from nltk.corpus import reuters

In [ ]:
## selecting subset of data for analysis

10788

# Dataset EDA

In [ ]:
print(f"Reuters corpus has {len(reuters.fileids())} news articles")
print(f"Reuters corpus has {len(reuters.categories())} categories")

Reuters corpus has 10788 news articles
Reuters corpus has 90 categories
